In [3]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
tqdm.pandas()
with open('/Users/janechoi/Desktop/2020-CO-DEEP/PREPROCESS/newcrawl/new_crawl_list.pickle', 'rb') as f:
    book_id = pickle.load(f)

In [9]:
## BEFORE YOU START... 
#1. install libraries
#2. install chromedriver >> reference :: (https://beomi.github.io/gb-crawling/posts/2017-02-27-HowToMakeWebCrawler-With-Selenium.html)
#3. make a new folder for save book covers

import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
# !pip install selenium
from selenium import webdriver
import urllib

import pandas as pd 
import numpy as np

import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

# os.chdir("/Users/janechoi/Desktop/2020-CO-DEEP/crawling-12")

# data.head()

/content


In [19]:
jy = book_id[8000:16000]
jy[:10]
len(jy)
jy[0]


WebDriverException: ignored

In [ ]:
bookdb = pd.DataFrame()
    
driver = webdriver.Chrome('/Users/janechoi/Desktop/2020-CO-DEEP/PREPROCESS/crawling-12/chromedriver') 

driver.implicitly_wait(3)
      
for i in tqdm(range(17795,20000)):
    try:
        time.sleep(1)
        book_id = jy[i]
  
        url = "https://lib.skku.edu/#/search/total/si?all=1%7Ck%7Ca%7C{}".format(book_id)
        driver.get(url)
    
    
        result = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/ik-total-search[1]/div/div[2]/div/div[2]/ul/li[1]/a[2]')
        result.click()
    
        driver.switch_to.window(driver.window_handles[1])
    
    #open the detailed info 
        try:
            driver.find_element_by_css_selector('#btn-biblio-more-open').click()
        except: 
            pass

    
    #image retreival 
        book_cover = driver.find_elements_by_css_selector('div.ikc-bookcover')
        img_url = re.findall('(?<=src=").+?(?=")', book_cover[0].get_attribute('outerHTML'))[0]
    
        try :
            img_name = 'cover/' + book_id + '.jpg'
            urllib.request.urlretrieve(img_url,img_name)
            img = 'Y'
        
        except:
            img = 'N'
    
    
    
    ##book title 
        book_title = driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child(1) > span').text

        title = book_title.split('/')[0]
    
        for i in range(3,9):
            try:
                text= driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child('+str(i)+') > label').text
       
        ##book pub year 
                if text == '발행사항':
                    book_pub= driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child('+str(i)+') > span > span').text
                
                    pubyear= book_pub.split(',')[-1]
#                 print(pubyear)
        
        ## book page 
                elif text == '형태사항':
                    book_page= driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child('+str(i)+') > span > span').text
                
                    page= book_page.split(' p.')[0]
#                 print(page)
    
                else: 
                    pass
            except:
                break
    
        if pubyear == None: 
            pubyear= np.nan
    
        if page == None:
            page = np.nan
    
        bookdb_new = bookdb_new.append({"book_id" : book_id, "subject" : title, "pub_year" : pubyear, 
    
                                        "page" : page, 'img' : img}, ignore_index=True)
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
    
    except:
        bookdb_new =bookdb_new.append({"book_id" : book_id, 'subject' : 'notfound'
                                       ,'pub_year' : 'notfound', 'page': 'notfound',
                                      'img':'notfound'}, ignore_index=True)
        
    
    
    bookdb_new.to_csv('new_books_jinhang.csv', index = False)